# Care Team Engagment Prediction - Data preprocessing

This notebook includes the following steps:
<ul>
<li> Setup
<li> Read data file </li>
<li> Process data file </li>
<li> Remove nulls </li>
<li> Define Y </li>
</ul>

### 0. Setup

In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [110]:
%matplotlib inline

### 1. Read data file (locally or from S3) 

In [111]:
# Read local data file
df1 = pd.read_csv('data5.csv')

### 2. Process Input file

In [112]:
df1.shape

(53895, 30)

In [113]:
df1.columns

Index(['days_to_first_et', 'days_to_coach', 'indication', 'is_gender_female',
       'is_gender_male', 'is_gender_other', 'bio_length', 'reasons_length',
       'imagine_free_length', 'reason_limited_time',
       'reason_family_obligations', 'reason_work_obligations', 'reason_other',
       'surgery_1yr', 'pain_severity', 'pain_vas', 'pain_description_length',
       'bmi', 'gad', 'phq', 'inbound_coach_messages_4_weeks',
       'inbound_coach_messages_1_week', 'inbound_coach_messages_length_1_week',
       'inbound_member_messages_4_weeks', 'inbound_member_messages_1_week',
       'surgery_message', 'call_message', 'interaction_message',
       'video_message', 'booking_message'],
      dtype='object')

In [114]:
# Replace indication with dummy variables
indication_dummies = pd.get_dummies(df1['indication'])
df1 = pd.concat([df1, indication_dummies], axis=1)      
df1 = df1.drop(['indication'], axis=1)

In [115]:
# Combine all hot words into one column
df1['hot_word'] =  df1['surgery_message'] + df1['call_message'] + df1['interaction_message'] + df1['video_message']

# take out of the members that used a hot word. We know those should be assign to mid level
hot_word_memebrs = df1[df1['hot_word'] > 0]


### 3. Remove null values

In [116]:
print(df1.isnull().sum()) # found no missing values in the data

days_to_first_et                            0
days_to_coach                            1010
is_gender_female                            0
is_gender_male                              0
is_gender_other                             0
bio_length                                  0
reasons_length                              0
imagine_free_length                         0
reason_limited_time                         0
reason_family_obligations                   0
reason_work_obligations                     0
reason_other                                0
surgery_1yr                                 0
pain_severity                           12466
pain_vas                                 2174
pain_description_length                     0
bmi                                      2377
gad                                         6
phq                                         7
inbound_coach_messages_4_weeks              0
inbound_coach_messages_1_week               0
inbound_coach_messages_length_1_we

In [117]:
# Remove members with transferred_to_coach_day = null
df1 = df1[df1['days_to_coach'].notna()]

In [118]:
# if pain_severity or pain_vas is null -> 0
df1['pain_severity'].fillna(0, inplace=True)
df1['pain_vas'].fillna(0, inplace=True)
df1['gad'].fillna(0, inplace=True)
df1['phq'].fillna(0, inplace=True)

In [119]:
# but average BMI where BMI is null
df1['bmi'].fillna((df1['bmi'].mean()), inplace=True)

In [120]:
print(df1.isnull().sum()) # found no missing values in the data

days_to_first_et                        0
days_to_coach                           0
is_gender_female                        0
is_gender_male                          0
is_gender_other                         0
bio_length                              0
reasons_length                          0
imagine_free_length                     0
reason_limited_time                     0
reason_family_obligations               0
reason_work_obligations                 0
reason_other                            0
surgery_1yr                             0
pain_severity                           0
pain_vas                                0
pain_description_length                 0
bmi                                     0
gad                                     0
phq                                     0
inbound_coach_messages_4_weeks          0
inbound_coach_messages_1_week           0
inbound_coach_messages_length_1_week    0
inbound_member_messages_4_weeks         0
inbound_member_messages_1_week    

### 4. Define Y

In [121]:
limit = 7

# Define target column
# See analysis below showed the 20% of customer = 9 or more messages
df1['Y'] = df1['inbound_coach_messages_4_weeks'] > limit


In [122]:
df1['Y'].value_counts()

False    40402
True     12483
Name: Y, dtype: int64

### 5. Save output file

In [123]:
# Save cleaned data for future use
df1.to_csv('analysisData1.csv', index=False)

In [124]:
# Remove members that used hot words as we know they are mid-tier
df1 = df1[df1['hot_word'] == 0] 

# Remove not needed columns
df1 = df1.drop(['inbound_member_messages_4_weeks', 'inbound_coach_messages_4_weeks'], axis=1)
df1 = df1.drop(['surgery_message', 'call_message', 'interaction_message', 'video_message', 'booking_message', 'hot_word'], axis=1)

# Save file
df1.to_csv('cleanData1.csv', index=False)